# 4.1.1 角點檢測 (Corner Detection)

**WBS 4.1.1**: 角點檢測基礎與應用

本模組涵蓋:
- **Harris 角點檢測**: 經典角點檢測算法
- **Shi-Tomasi 角點檢測**: 改進的 Harris 算法
- **FAST 角點檢測**: 快速特徵檢測算法
- **角點檢測器比較分析**: 性能與應用場景評估

<div style="page-break-after: always"></div>

# 1. 什麼是角點 (Corner)?

## 1-1: 角點的定義

> 角點是圖像中**局部特徵最明顯**的點，具有以下特性:
> * **兩個邊緣的交叉點**: 圖像中兩條邊緣線的交匯處
> * **灰度梯度變化劇烈**: 在多個方向上灰度變化都很大
> * **穩定特徵**: 具有旋轉不變性，對光照變化相對穩定
> * **可重複檢測**: 在不同視角下仍能被檢測到

## 1-2: 角點檢測的重要性

> 角點檢測是計算機視覺中的**基礎操作**，廣泛應用於:
> * **特徵匹配**: 兩張圖像之間的對應點匹配
> * **運動估計**: 視頻序列中的物體追蹤
> * **3D 重建**: 從多視角圖像重建三維模型
> * **物體識別**: 識別和定位場景中的物體
> * **影像拼接**: 全景圖像生成

## 1-3: 角點 vs. 邊緣

> | 特徵 | 角點 (Corner) | 邊緣 (Edge) |
> |------|--------------|-------------|
> | 定義 | 兩個邊緣的交點 | 圖像灰度劇烈變化的地方 |
> | 方向性 | **多方向**梯度變化 | **單一方向**梯度變化 |
> | 定位精度 | **精確定位** | 相對模糊 |
> | 穩定性 | **高穩定性** | 容易受噪聲影響 |
> | 數量 | 少量關鍵點 | 大量邊緣點 |

### 視覺化示意

> **平坦區域 (Flat Region)**: 各方向移動，灰度變化都很小  
> **邊緣 (Edge)**: 垂直邊緣方向移動，灰度變化大  
> **角點 (Corner)**: 任意方向移動，灰度變化都很大

><img src="../assets/images/basic/corner_concept.png" style='width:80%'></img>

<div style="page-break-after: always"></div>

# 2. Harris 角點檢測

## 2-1: Harris 算法原理

> Harris 角點檢測是由 **Chris Harris** 和 **Mike Stephens** 於 1988 年提出的經典算法。

### 核心思想

> 通過計算圖像梯度的**自相關矩陣 (Autocorrelation Matrix)** 來檢測角點:
> 
> 1. 計算圖像在 x 和 y 方向的梯度 (使用 Sobel 算子)
> 2. 構建自相關矩陣 M
> 3. 計算角點響應函數 R
> 4. 根據閾值判斷是否為角點

### 數學公式

> **自相關矩陣 M**:
>
> $$M = \sum_{x,y} w(x,y) \begin{bmatrix} I_x^2 & I_x I_y \\ I_x I_y & I_y^2 \end{bmatrix}$$
>
> 其中:
> * $I_x, I_y$ 是圖像的梯度
> * $w(x,y)$ 是高斯窗口函數

> **Harris 角點響應函數 R**:
>
> $$R = det(M) - k \cdot trace(M)^2$$
> $$R = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2$$
>
> 其中:
> * $\lambda_1, \lambda_2$ 是矩陣 M 的特徵值
> * $k$ 是經驗常數 (通常取 0.04 ~ 0.06)

### 判斷準則

> | 情況 | $\lambda_1$ | $\lambda_2$ | R 值 | 區域類型 |
> |------|------------|------------|------|----------|
> | 1 | 小 | 小 | 小 | 平坦區域 |
> | 2 | 大 | 小 | 負數 | 邊緣 |
> | 3 | 大 | 大 | **大正數** | **角點** |

## 2-2: OpenCV 實現

### cv2.cornerHarris() 函數

```python
dst = cv2.cornerHarris(src, blockSize, ksize, k)
```

> **參數說明**:
> * `src`: 輸入灰度圖像 (float32 類型)
> * `blockSize`: 角點檢測的鄰域大小 (通常為 2~5)
> * `ksize`: Sobel 算子的孔徑大小 (必須是奇數)
> * `k`: Harris 檢測器的自由參數 (0.04 ~ 0.06)
>
> **返回值**:
> * `dst`: 每個像素的角點響應值 (float32 類型)

In [ ]:
# 導入必要的庫
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2-3: Harris 角點檢測實作

In [ ]:
# Load test image
img = cv2.imread('../assets/images/basic/lenaColor.png', 1)
if img is None:
    print("Error: Could not load image")
else:
    print(f"Image loaded: {img.shape}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)  # Harris requires float32 type
    
    # Apply Harris corner detection
    # blockSize: neighborhood size for corner detection
    # ksize: aperture parameter for Sobel operator
    # k: Harris detector free parameter (0.04-0.06)
    dst = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.04)
    
    # Result is dilated for marking the corners (optional enhancement)
    dst = cv2.dilate(dst, None)
    
    # Create visualization
    img_display = img.copy()
    
    # Threshold: mark corners where response > 0.01 * max response
    img_display[dst > 0.01 * dst.max()] = [0, 0, 255]  # Red color for corners
    
    # Display results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(dst, cmap='hot')
    axes[1].set_title('Harris Response (Corner Strength)')
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))
    axes[2].set_title('Detected Corners (Red)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    corner_count = np.sum(dst > 0.01 * dst.max())
    print(f"\nDetected corners: {corner_count}")
    print(f"Max response: {dst.max():.2f}")
    print(f"Threshold: {0.01 * dst.max():.2f}")

### 參數調整影響

In [ ]:
# Compare different Harris parameters
if img is not None:
    gray = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    
    # Test different k values
    k_values = [0.04, 0.06, 0.08]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    for idx, k in enumerate(k_values):
        # Apply Harris corner detection
        dst = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=k)
        dst = cv2.dilate(dst, None)
        
        # Visualize response
        axes[0, idx].imshow(dst, cmap='hot')
        axes[0, idx].set_title(f'Response (k={k})')
        axes[0, idx].axis('off')
        
        # Mark corners
        img_marked = img.copy()
        img_marked[dst > 0.01 * dst.max()] = [0, 0, 255]
        
        axes[1, idx].imshow(cv2.cvtColor(img_marked, cv2.COLOR_BGR2RGB))
        axes[1, idx].set_title(f'Corners (k={k}, count={np.sum(dst > 0.01 * dst.max())})')
        axes[1, idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Parameter k 影響:")
    print("- k 越小: 檢測到更多角點 (包含弱角點)")
    print("- k 越大: 只檢測強角點 (更嚴格的判斷標準)")

<div style="page-break-after: always"></div>

# 3. Shi-Tomasi 角點檢測

## 3-1: Shi-Tomasi 算法原理

> Shi-Tomasi 算法是 **J. Shi** 和 **C. Tomasi** 於 1994 年提出的，是對 Harris 算法的改進。

### 改進之處

> **Harris 角點響應函數**:
> $$R = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2$$
>
> **Shi-Tomasi 角點響應函數** (更簡單):
> $$R = min(\lambda_1, \lambda_2)$$

### 優勢

> * **無需調整參數 k**: 去除了 Harris 中的經驗參數
> * **更穩定**: 使用最小特徵值作為判斷標準
> * **更適合追蹤**: 廣泛用於光流法和特徵追蹤
> * **可指定數量**: 可以指定檢測最好的 N 個角點

## 3-2: OpenCV 實現

### cv2.goodFeaturesToTrack() 函數

```python
corners = cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance)
```

> **參數說明**:
> * `image`: 輸入灰度圖像 (8-bit or float32)
> * `maxCorners`: 返回角點的最大數量 (0 表示無限制)
> * `qualityLevel`: 角點質量水平 (0~1)，低於 `qualityLevel * max(response)` 的角點被拒絕
> * `minDistance`: 角點之間的最小歐氏距離
>
> **可選參數**:
> * `mask`: 掩碼，指定檢測區域
> * `blockSize`: 計算協方差矩陣的窗口大小
> * `useHarrisDetector`: 是否使用 Harris 檢測器 (False 使用 Shi-Tomasi)
> * `k`: Harris 檢測器的 k 參數
>
> **返回值**:
> * `corners`: 檢測到的角點座標 array，shape 為 (N, 1, 2)

## 3-3: Shi-Tomasi 角點檢測實作

In [ ]:
# Shi-Tomasi corner detection
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect corners using Shi-Tomasi method
    # maxCorners: maximum number of corners to return
    # qualityLevel: parameter characterizing the minimal accepted quality
    # minDistance: minimum possible Euclidean distance between returned corners
    corners = cv2.goodFeaturesToTrack(
        gray,
        maxCorners=100,
        qualityLevel=0.01,
        minDistance=10,
        blockSize=7
    )
    
    # Draw detected corners
    img_corners = img.copy()
    
    if corners is not None:
        corners = np.int0(corners)  # Convert to integer
        
        for i, corner in enumerate(corners):
            x, y = corner.ravel()  # Extract x, y coordinates
            # Draw circle at corner location
            cv2.circle(img_corners, (x, y), 5, (0, 255, 0), -1)
    
    # Display results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_corners, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'Shi-Tomasi Corners (Green, count={len(corners)})')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print corner statistics
    print(f"\nDetected {len(corners)} corners")
    print(f"Corner coordinates shape: {corners.shape}")
    print(f"First 5 corners:\n{corners[:5].reshape(-1, 2)}")

### 參數調整對比

In [ ]:
# Compare different parameters for Shi-Tomasi
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Test different parameter combinations
    params = [
        {'maxCorners': 50, 'qualityLevel': 0.01, 'minDistance': 10},
        {'maxCorners': 100, 'qualityLevel': 0.01, 'minDistance': 10},
        {'maxCorners': 100, 'qualityLevel': 0.05, 'minDistance': 10},
        {'maxCorners': 100, 'qualityLevel': 0.01, 'minDistance': 20}
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.ravel()
    
    for idx, param in enumerate(params):
        # Detect corners
        corners = cv2.goodFeaturesToTrack(gray, **param)
        
        # Draw corners
        img_marked = img.copy()
        if corners is not None:
            corners = np.int0(corners)
            for corner in corners:
                x, y = corner.ravel()
                cv2.circle(img_marked, (x, y), 5, (0, 255, 0), -1)
        
        # Display
        axes[idx].imshow(cv2.cvtColor(img_marked, cv2.COLOR_BGR2RGB))
        title = f"max={param['maxCorners']}, q={param['qualityLevel']}, d={param['minDistance']}\n"
        title += f"Detected: {len(corners) if corners is not None else 0}"
        axes[idx].set_title(title)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 參數影響分析:")
    print("- maxCorners: 限制返回的最大角點數")
    print("- qualityLevel: 越高越嚴格，檢測到的角點越少但質量越高")
    print("- minDistance: 角點之間的最小距離，避免角點聚集")

<div style="page-break-after: always"></div>

# 4. FAST 角點檢測

## 4-1: FAST 算法原理

> **FAST** (Features from Accelerated Segment Test) 由 **Edward Rosten** 和 **Tom Drummond** 於 2006 年提出。

### 核心思想

> FAST 的設計目標是**速度**，特別適合即時應用。原理簡單高效:
>
> 1. 選擇圖像中的一個像素 p，其灰度值為 $I_p$
> 2. 設定閾值 t (例如 $I_p$ 的 20%)
> 3. 考慮以 p 為中心、半徑為 3 的圓，圓上有 16 個像素點
> 4. 如果圓上有**連續 n 個點** (通常 n=12，稱為 FAST-12):
>    - 這些點的灰度值都 > $I_p + t$ (更亮)
>    - 或都 < $I_p - t$ (更暗)
> 5. 則認為 p 是一個角點

### FAST 加速技巧

> 為了更快的速度，FAST 使用**快速判斷策略**:
> * 首先檢查圓上的 4 個點 (1, 5, 9, 13) - 每隔 90 度
> * 如果其中至少有 3 個點滿足條件，才繼續檢查其他點
> * 否則直接放棄該候選點

><img src="../assets/images/basic/KpFast01.png" style='width:80%'></img>

### FAST 的優缺點

> **優點**:
> * ✅ **執行速度極快**: 適合即時應用和低階硬體
> * ✅ **原理簡單**: 易於實現和理解
> * ✅ **使用率高**: 廣泛應用於 SLAM、AR 等領域
>
> **缺點**:
> * ❌ **角點聚集**: 檢測到的角點容易"扎堆" (需要非極大值抑制)
> * ❌ **無方向資訊**: 不具有旋轉不變性
> * ❌ **無尺度資訊**: 不具有尺度不變性
> * ❌ **無描述子**: 僅檢測角點，不包含特徵描述

### 非極大值抑制 (NMS)

> 為了解決角點聚集問題，FAST 使用 **Non-Maximum Suppression (NMS)**:
> * 計算每個候選角點的響應值
> * 在鄰域內只保留響應值最大的角點
> * 抑制其他較弱的角點

## 4-2: OpenCV 實現

### cv2.FastFeatureDetector 類

```python
fast = cv2.FastFeatureDetector_create(
    threshold=10,
    nonmaxSuppression=True,
    type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16
)
keypoints = fast.detect(image, None)
```

> **參數說明**:
> * `threshold`: 中心像素與圓上像素的差異閾值
> * `nonmaxSuppression`: 是否進行非極大值抑制
> * `type`: FAST 類型
>   - `TYPE_5_8`: 圓上 8 個點，需要連續 5 個
>   - `TYPE_7_12`: 圓上 12 個點，需要連續 7 個
>   - `TYPE_9_16`: 圓上 16 個點，需要連續 9 個 (預設)
>
> **返回值**:
> * `keypoints`: 檢測到的關鍵點列表，每個關鍵點包含位置、大小、角度等資訊

## 4-3: FAST 角點檢測實作

In [ ]:
# FAST feature detection
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Create FAST detector
    # threshold: threshold on difference between intensity of central pixel and pixels on circle
    # nonmaxSuppression: if true, non-maximum suppression is applied to detected corners
    fast = cv2.FastFeatureDetector_create(
        threshold=20,
        nonmaxSuppression=True,
        type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16
    )
    
    # Detect keypoints
    keypoints = fast.detect(gray, None)
    
    # Draw keypoints
    img_kp = cv2.drawKeypoints(
        img,
        keypoints,
        None,
        color=(0, 255, 0),
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    )
    
    # Display results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'FAST Keypoints (count={len(keypoints)})')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print keypoint information
    print(f"\nTotal keypoints detected: {len(keypoints)}")
    print(f"\nFirst 5 keypoints information:")
    for i, kp in enumerate(keypoints[:5]):
        print(f"  KP {i}: pt=({kp.pt[0]:.1f}, {kp.pt[1]:.1f}), size={kp.size:.1f}, response={kp.response:.3f}")

### FAST 參數影響對比

In [ ]:
# Compare FAST with and without Non-Maximum Suppression
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # FAST without NMS
    fast_no_nms = cv2.FastFeatureDetector_create(
        threshold=20,
        nonmaxSuppression=False
    )
    kp_no_nms = fast_no_nms.detect(gray, None)
    img_no_nms = cv2.drawKeypoints(img, kp_no_nms, None, color=(0, 255, 0))
    
    # FAST with NMS
    fast_nms = cv2.FastFeatureDetector_create(
        threshold=20,
        nonmaxSuppression=True
    )
    kp_nms = fast_nms.detect(gray, None)
    img_nms = cv2.drawKeypoints(img, kp_nms, None, color=(0, 255, 0))
    
    # Display comparison
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(cv2.cvtColor(img_no_nms, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Without NMS (count={len(kp_no_nms)})')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_nms, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'With NMS (count={len(kp_nms)})')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n非極大值抑制 (NMS) 效果:")
    print(f"  Without NMS: {len(kp_no_nms)} keypoints (可能有聚集現象)")
    print(f"  With NMS: {len(kp_nms)} keypoints (去除冗餘角點)")
    print(f"  減少比例: {(1 - len(kp_nms)/len(kp_no_nms))*100:.1f}%")

### FAST 不同閾值對比

In [ ]:
# Compare different FAST thresholds
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    thresholds = [10, 20, 40, 80]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.ravel()
    
    for idx, thresh in enumerate(thresholds):
        # Create FAST detector with different threshold
        fast = cv2.FastFeatureDetector_create(
            threshold=thresh,
            nonmaxSuppression=True
        )
        
        # Detect and draw keypoints
        keypoints = fast.detect(gray, None)
        img_kp = cv2.drawKeypoints(img, keypoints, None, color=(0, 255, 0))
        
        # Display
        axes[idx].imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f'Threshold={thresh}, Keypoints={len(keypoints)}')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Threshold 影響分析:")
    print("- Threshold 越小: 檢測到更多角點 (包含弱特徵)")
    print("- Threshold 越大: 只檢測強角點 (更嚴格的標準)")
    print("- 建議根據應用場景調整: 即時追蹤用較低值，精確匹配用較高值")

<div style="page-break-after: always"></div>

# 5. 角點檢測器比較分析

## 5-1: 三種檢測器視覺對比

In [ ]:
# Comprehensive comparison of all three corner detectors
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_float = np.float32(gray)
    
    # ===== 1. Harris Corner Detection =====
    harris_dst = cv2.cornerHarris(gray_float, blockSize=5, ksize=3, k=0.04)
    harris_dst = cv2.dilate(harris_dst, None)
    img_harris = img.copy()
    img_harris[harris_dst > 0.01 * harris_dst.max()] = [255, 0, 0]  # Blue
    harris_count = np.sum(harris_dst > 0.01 * harris_dst.max())
    
    # ===== 2. Shi-Tomasi Corner Detection =====
    shi_tomasi_corners = cv2.goodFeaturesToTrack(
        gray,
        maxCorners=100,
        qualityLevel=0.01,
        minDistance=10
    )
    img_shi_tomasi = img.copy()
    if shi_tomasi_corners is not None:
        shi_tomasi_corners = np.int0(shi_tomasi_corners)
        for corner in shi_tomasi_corners:
            x, y = corner.ravel()
            cv2.circle(img_shi_tomasi, (x, y), 5, (0, 255, 0), -1)  # Green
    shi_tomasi_count = len(shi_tomasi_corners) if shi_tomasi_corners is not None else 0
    
    # ===== 3. FAST Feature Detection =====
    fast = cv2.FastFeatureDetector_create(threshold=20, nonmaxSuppression=True)
    fast_keypoints = fast.detect(gray, None)
    img_fast = img.copy()
    for kp in fast_keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        cv2.circle(img_fast, (x, y), 3, (0, 0, 255), -1)  # Red
    fast_count = len(fast_keypoints)
    
    # Display comparison
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(img_harris, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title(f'Harris (Blue, {harris_count} corners)', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(cv2.cvtColor(img_shi_tomasi, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title(f'Shi-Tomasi (Green, {shi_tomasi_count} corners)', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cv2.cvtColor(img_fast, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title(f'FAST (Red, {fast_count} keypoints)', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\n" + "="*60)
    print("角點檢測器統計比較")
    print("="*60)
    print(f"Harris      : {harris_count} 個角點")
    print(f"Shi-Tomasi  : {shi_tomasi_count} 個角點")
    print(f"FAST        : {fast_count} 個關鍵點")
    print("="*60)

## 5-2: 性能基準測試 (Performance Benchmarking)

In [ ]:
import time

# Performance benchmarking
if img is not None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_float = np.float32(gray)
    
    num_runs = 10
    
    # ===== Benchmark Harris =====
    start = time.time()
    for _ in range(num_runs):
        harris_dst = cv2.cornerHarris(gray_float, blockSize=5, ksize=3, k=0.04)
        harris_dst = cv2.dilate(harris_dst, None)
    harris_time = (time.time() - start) / num_runs * 1000  # Convert to ms
    
    # ===== Benchmark Shi-Tomasi =====
    start = time.time()
    for _ in range(num_runs):
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.01, minDistance=10)
    shi_tomasi_time = (time.time() - start) / num_runs * 1000
    
    # ===== Benchmark FAST =====
    fast = cv2.FastFeatureDetector_create(threshold=20, nonmaxSuppression=True)
    start = time.time()
    for _ in range(num_runs):
        keypoints = fast.detect(gray, None)
    fast_time = (time.time() - start) / num_runs * 1000
    
    # Visualization
    methods = ['Harris', 'Shi-Tomasi', 'FAST']
    times = [harris_time, shi_tomasi_time, fast_time]
    colors = ['#3498db', '#2ecc71', '#e74c3c']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(methods, times, color=colors, alpha=0.7, edgecolor='black')
    
    # Add value labels on bars
    for bar, time_val in zip(bars, times):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{time_val:.2f} ms',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('執行時間 (milliseconds)', fontsize=12, fontweight='bold')
    ax.set_title(f'角點檢測器性能比較 (平均 {num_runs} 次執行)', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print("\n" + "="*60)
    print("性能基準測試結果")
    print("="*60)
    print(f"圖像大小: {gray.shape}")
    print(f"測試次數: {num_runs}")
    print("-"*60)
    print(f"Harris      : {harris_time:.2f} ms (基準)")
    print(f"Shi-Tomasi  : {shi_tomasi_time:.2f} ms ({shi_tomasi_time/harris_time:.2f}x)")
    print(f"FAST        : {fast_time:.2f} ms ({fast_time/harris_time:.2f}x) ⚡")
    print("="*60)
    print(f"\n🏆 最快: {methods[np.argmin(times)]} ({min(times):.2f} ms)")
    print(f"💡 FAST 比 Harris 快 {harris_time/fast_time:.1f} 倍")

## 5-3: 綜合特性比較表

| 特性 | Harris | Shi-Tomasi | FAST |
|------|--------|------------|------|
| **檢測速度** | 中等 | 中等 | ⚡ **極快** |
| **檢測精度** | 高 | **最高** | 中等 |
| **旋轉不變性** | ✅ 是 | ✅ 是 | ❌ 否 |
| **尺度不變性** | ❌ 否 | ❌ 否 | ❌ 否 |
| **抗噪能力** | 好 | 好 | 中等 |
| **參數調整** | 需要 k 參數 | ✅ 簡單 | 需要閾值 |
| **角點分佈** | 均勻 | **最均勻** | 容易聚集 |
| **特徵描述子** | 無 | 無 | 無 (需配合其他) |
| **適用場景** | 一般特徵檢測 | 特徵追蹤、光流 | 即時視頻、SLAM |

### 算法複雜度分析

| 算法 | 時間複雜度 | 空間複雜度 | 備註 |
|------|-----------|-----------|------|
| **Harris** | O(n²) | O(n²) | n 為圖像尺寸 |
| **Shi-Tomasi** | O(n²) | O(n²) | 與 Harris 相似 |
| **FAST** | O(n) | O(1) | 線性時間，常數空間 |

## 5-4: 不同場景下的表現

In [ ]:
# Test on different types of images
test_images = [
    '../assets/images/basic/lenaColor.png',
    '../assets/images/basic/chessboard.png',
    '../assets/images/basic/building.jpg'
]

image_names = ['Lena (紋理豐富)', 'Chessboard (規則圖案)', 'Building (邊緣明顯)']

results = []

for idx, img_path in enumerate(test_images):
    test_img = cv2.imread(img_path)
    if test_img is None:
        print(f"Warning: Could not load {img_path}")
        continue
    
    # Resize if too large
    if max(test_img.shape[:2]) > 512:
        scale = 512 / max(test_img.shape[:2])
        test_img = cv2.resize(test_img, None, fx=scale, fy=scale)
    
    gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    gray_float = np.float32(gray)
    
    # Harris
    harris_dst = cv2.cornerHarris(gray_float, 5, 3, 0.04)
    harris_count = np.sum(harris_dst > 0.01 * harris_dst.max())
    
    # Shi-Tomasi
    corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)
    shi_count = len(corners) if corners is not None else 0
    
    # FAST
    fast = cv2.FastFeatureDetector_create(20, True)
    kps = fast.detect(gray, None)
    fast_count = len(kps)
    
    results.append([harris_count, shi_count, fast_count])

if results:
    # Plot comparison
    results = np.array(results)
    x = np.arange(len(image_names))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars1 = ax.bar(x - width, results[:, 0], width, label='Harris', color='#3498db', alpha=0.8)
    bars2 = ax.bar(x, results[:, 1], width, label='Shi-Tomasi', color='#2ecc71', alpha=0.8)
    bars3 = ax.bar(x + width, results[:, 2], width, label='FAST', color='#e74c3c', alpha=0.8)
    
    ax.set_xlabel('圖像類型', fontsize=12, fontweight='bold')
    ax.set_ylabel('檢測到的角點數量', fontsize=12, fontweight='bold')
    ax.set_title('不同場景下的角點檢測表現', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(image_names)
    ax.legend(fontsize=11)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.show()
    
    print("\n不同場景檢測結果:")
    print("-" * 60)
    for name, result in zip(image_names, results):
        print(f"{name:20} | Harris: {result[0]:4d} | Shi-Tomasi: {result[1]:4d} | FAST: {result[2]:4d}")

<div style="page-break-after: always"></div>

# 6. 應用場景與選擇建議

## 6-1: 應用場景分析

### Harris 角點檢測

> **適用場景**:
> * ✅ 靜態圖像特徵提取
> * ✅ 圖像拼接 (panorama stitching)
> * ✅ 3D 重建預處理
> * ✅ 圖像配準 (image registration)
>
> **不適用場景**:
> * ❌ 即時視頻處理 (速度較慢)
> * ❌ 需要尺度不變性的應用
> * ❌ 低端硬體設備

### Shi-Tomasi 角點檢測

> **適用場景**:
> * ✅ **光流法追蹤** (Lucas-Kanade 光流) ⭐
> * ✅ 視頻物體追蹤
> * ✅ 需要高質量角點的應用
> * ✅ 特徵點匹配
> * ✅ 相機標定
>
> **典型應用**:
> * OpenCV 的 `calcOpticalFlowPyrLK()` 預設使用 Shi-Tomasi
> * 視頻穩定 (video stabilization)
> * 運動分析

### FAST 角點檢測

> **適用場景**:
> * ✅ **即時視頻處理** ⭐
> * ✅ **SLAM (同步定位與地圖構建)** ⭐
> * ✅ **AR/VR 應用** ⭐
> * ✅ 移動設備應用
> * ✅ 無人機視覺導航
> * ✅ 低端硬體設備
>
> **典型應用**:
> * ORB-SLAM (使用 ORB = Oriented FAST + BRIEF)
> * 即時目標追蹤
> * 移動機器人視覺

## 6-2: 選擇決策樹

```
需要即時處理 (>30 FPS)?
├─ 是 → 使用 FAST
└─ 否
    └─ 需要追蹤特徵點?
        ├─ 是 → 使用 Shi-Tomasi
        └─ 否 → 使用 Harris 或 Shi-Tomasi
```

## 6-3: 實用建議

### 參數調整建議

| 算法 | 關鍵參數 | 建議範圍 | 調整策略 |
|------|---------|---------|----------|
| **Harris** | k | 0.04 ~ 0.06 | 角點太少增加 k，太多減少 k |
| **Harris** | threshold | 0.01 ~ 0.1 | 相對於最大響應值的比例 |
| **Shi-Tomasi** | qualityLevel | 0.01 ~ 0.1 | 質量越高角點越少 |
| **Shi-Tomasi** | minDistance | 5 ~ 30 | 避免角點聚集 |
| **FAST** | threshold | 10 ~ 40 | 閾值越高角點越少 |
| **FAST** | NMS | True/False | 建議開啟以避免聚集 |

### 性能優化建議

> **1. 圖像預處理**:
> * 適當縮小圖像尺寸 (如 640x480)
> * 使用高斯模糊去除噪聲
> * 增強對比度 (對於低對比度圖像)
>
> **2. 區域限制**:
> * 使用 mask 限制檢測區域
> * 排除不需要檢測的區域 (如天空)
>
> **3. 多尺度檢測**:
> * 構建圖像金字塔
> * 在不同尺度上檢測角點
> * 適用於需要尺度不變性的應用

## 6-4: 進階技術

### 角點檢測的改進算法

> **1. ORB (Oriented FAST and Rotated BRIEF)**:
> * 基於 FAST 的改進
> * 添加方向信息 (旋轉不變性)
> * 使用 BRIEF 描述子
>
> **2. BRISK (Binary Robust Invariant Scalable Keypoints)**:
> * 具有尺度不變性
> * 速度接近 FAST
> * 適合即時應用
>
> **3. AGAST (Adaptive and Generic Accelerated Segment Test)**:
> * FAST 的改進版本
> * 更快的檢測速度
> * 更好的檢測質量

## 6-5: 實戰案例 - 特徵追蹤

In [ ]:
# Practical example: Feature tracking using Shi-Tomasi + Optical Flow
# This demonstrates why Shi-Tomasi is preferred for tracking applications

print("\n" + "="*60)
print("實戰案例: Shi-Tomasi 角點 + Lucas-Kanade 光流追蹤")
print("="*60)
print("\n應用場景: 視頻物體追蹤、運動分析")
print("\n算法流程:")
print("  1. 使用 Shi-Tomasi 檢測第一幀的角點")
print("  2. 使用 Lucas-Kanade 光流追蹤角點在後續幀中的位置")
print("  3. 根據追蹤結果分析物體運動")
print("\n為什麼選擇 Shi-Tomasi?")
print("  ✅ 角點質量高，適合長期追蹤")
print("  ✅ 分佈均勻，覆蓋整個圖像")
print("  ✅ OpenCV 光流函數的預設選擇")
print("  ✅ 追蹤穩定性好")
print("\n" + "="*60)

# Simplified code structure (actual implementation would require video input)
code_example = '''
# Feature tracking pseudo-code
frame1 = capture.read()
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Detect corners using Shi-Tomasi
corners = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.01, minDistance=10)

while True:
    frame2 = capture.read()
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Track corners using Lucas-Kanade optical flow
    new_corners, status, error = cv2.calcOpticalFlowPyrLK(gray1, gray2, corners, None)
    
    # Draw tracked features
    for i, (new, old) in enumerate(zip(new_corners, corners)):
        if status[i]:
            cv2.line(frame2, tuple(new), tuple(old), (0, 255, 0), 2)
    
    # Update for next iteration
    gray1 = gray2.copy()
    corners = new_corners
'''

print("\n範例程式碼:")
print(code_example)

<div style="page-break-after: always"></div>

# 7. 總結與延伸

## 7-1: 核心要點回顧

### 三種角點檢測器總結

> **Harris 角點檢測**:
> * 📌 經典算法，理論基礎扎實
> * 📌 使用自相關矩陣和響應函數
> * 📌 需要調整參數 k (0.04~0.06)
> * 📌 適合一般圖像處理任務
>
> **Shi-Tomasi 角點檢測**:
> * 📌 Harris 的改進版本
> * 📌 使用最小特徵值，無需參數 k
> * 📌 **最適合特徵追蹤** ⭐
> * 📌 OpenCV 光流法的首選
>
> **FAST 角點檢測**:
> * 📌 **速度最快** 的角點檢測算法 ⚡
> * 📌 基於圓形模板的像素比較
> * 📌 適合即時應用和低端硬體
> * 📌 缺乏方向和尺度信息

## 7-2: 選擇指南

| 應用需求 | 推薦算法 | 理由 |
|---------|---------|------|
| 圖像拼接 | Harris / Shi-Tomasi | 需要高質量角點 |
| 特徵追蹤 | **Shi-Tomasi** ⭐ | 最適合光流法 |
| 即時視頻 | **FAST** ⚡ | 速度最快 |
| SLAM | FAST + ORB | 速度 + 描述子 |
| 3D重建 | Harris / Shi-Tomasi | 精度優先 |
| 移動設備 | FAST | 計算資源有限 |

## 7-3: 延伸學習

### 後續模組預告

> **4.1.2 特徵描述子 (Feature Descriptors)**:
> * SIFT (Scale-Invariant Feature Transform)
> * SURF (Speeded-Up Robust Features)
> * ORB (Oriented FAST and Rotated BRIEF)
> * BRIEF (Binary Robust Independent Elementary Features)
>
> **4.1.3 特徵匹配 (Feature Matching)**:
> * Brute-Force 匹配
> * FLANN 快速匹配
> * 交叉驗證匹配
> * RANSAC 外點過濾
>
> **4.2 光流法 (Optical Flow)**:
> * Lucas-Kanade 光流
> * Farneback 稠密光流
> * 運動估計與物體追蹤

## 7-4: 參考資源

### 論文

> * **Harris Corner Detection**:  
>   C. Harris and M. Stephens. "A Combined Corner and Edge Detector." *Alvey Vision Conference*, 1988.
>
> * **Shi-Tomasi Corner Detection**:  
>   J. Shi and C. Tomasi. "Good Features to Track." *IEEE Conference on CVPR*, 1994.
>
> * **FAST Corner Detection**:  
>   E. Rosten and T. Drummond. "Machine Learning for High-Speed Corner Detection." *ECCV*, 2006.

### 線上資源

> * OpenCV 官方文檔: https://docs.opencv.org/
> * Harris Corner Detector Tutorial: https://docs.opencv.org/master/dc/d0d/tutorial_py_features_harris.html
> * Shi-Tomasi Corner Detector: https://docs.opencv.org/master/d4/d8c/tutorial_py_shi_tomasi.html
> * FAST Algorithm: https://docs.opencv.org/master/df/d0c/tutorial_py_fast.html

---

## 🎯 實作練習建議

1. **比較實驗**: 在不同類型圖像上測試三種算法，分析結果差異
2. **參數調優**: 嘗試不同參數組合，理解參數對結果的影響
3. **性能測試**: 在不同硬體上測試算法性能，記錄執行時間
4. **追蹤實現**: 結合 Shi-Tomasi 和光流法實現簡單的物體追蹤
5. **即時應用**: 使用 FAST 實現即時視頻角點檢測

---

**模組完成！繼續學習 4.1.2 特徵描述子模組，深入了解如何描述和匹配檢測到的角點。**